<a href="https://colab.research.google.com/github/kunalsinght/stance_detection_using_bert/blob/main/Bert_stance_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

modelName = "distilbert-base-uncased"
# model = AutoModelForSequenceClassification.from_pretrained(modelName)
model = AutoModelForSequenceClassification.from_pretrained(
    modelName,
    num_labels=3,
    output_attentions=False,
    output_hidden_states=False,
    )
tokenizer = AutoTokenizer.from_pretrained(modelName)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [16]:
#example working of tokenizer
tokens = tokenizer.tokenize("Hello bert model stance","Hello apple")
tokenIds = tokenizer.convert_tokens_to_ids(tokens)

In [17]:
tokens

['hello', 'bert', 'model', 'stance', 'hello', 'apple']

In [18]:
tokenIds

[7592, 14324, 2944, 11032, 7592, 6207]

In [19]:
t = tokenizer("Hello bert model stance","Hello apple")

In [20]:
t

{'input_ids': [101, 7592, 14324, 2944, 11032, 102, 7592, 6207, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
t= tokenizer.encode_plus(["Hello the bert model stance","yash there"])
t

{'input_ids': [101, 7592, 1996, 14324, 2944, 11032, 102, 8038, 4095, 2045, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
# import torch 
# from torch.utils.data import Dataset

In [23]:
# class StanceDataset(Dataset):
#   def __init__(self,encodings,labels):
#     self.encodings = encodings
#     self.labels = labels

#   def __getitem__(self,idx):
#     item = {key: torch.tensor(val[idx]) for key,val in self.encodings.item()}
#     item['labels']= torch.tensor(self.labels[idx])
#     return item

#   def __len__(self):
#     return len(self.labels)

In [24]:
# def tokenize_func(data):
#   return tokenizer(data['articleBody'],data['Headline'],padding='max_length',truncation=True,max_length=128)

In [25]:
import pandas as pd
import numpy as np
import torch 

In [26]:
df = pd.read_csv('./stance_all_1_4.csv')
df = df.sample(frac = 1)
df.head(10)

,Unnamed: 0,Body ID,articleBody,Headline,Stance
951,34350,1873,"'It's a nonsense', says official\n\nNorth Kore...",Schoolboy almost killed by electric shock clai...,2
2163,22906,1331,Ottawa shooting video shown by police\n\nThe R...,That Boko Haram Ceasefire in Nigeria Isn’t A C...,2
2376,9179,564,The gunman in a fatal shooting that rocked Ott...,Viral Video Of Homeless Man Spending $100 On F...,2
1632,10906,666,Homeland Security Secretary Jeh Johnson says I...,"Vice's Shane Smith drops $300,000 on Vegas dinner",2
2718,17468,1022,A man had his genitals chopped off and thrown ...,Jordan’s King May Participate Personally In IS...,2
9035,22102,1293,ISIS claimed Friday that a 26-year-old female ...,Macaulay Culkin: Death claims about former chi...,2
2182,2308,125,Islamic State militants and sympathisers are t...,"Non, le lancement du nouveau «Star Wars» ne se...",2
1685,22532,1301,Elusive graffiti artist Banksy’s cover was blo...,Cameron vows to hunt down IS 'monsters' after ...,2
4959,44568,2302,Online reports in jihadi and Palestinian forum...,Ottawa Gunman Identified As Michael Zehaf-Bibeau,2
2501,4013,226,WASHINGTON (AP) — U.S. airstrikes earlier this...,Obama administration DENIES congressman's clai...,2


In [27]:
df['Stance'].unique()

array([2, 1, 0])

In [28]:
len(df)

10000

In [29]:
articleBodies = df.articleBody.values
headlines = df.Headline.values
stances = df.Stance.values

In [30]:
input_ids=[]
attention_masks=[]

for i in range(len(headlines)): 
  encoded_dict=tokenizer.encode_plus(
      [headlines[i],articleBodies[i]], #sentence to encode
      add_special_tokens=True, #add special characters
      max_length=256,
      truncation=True, #add max len and truncate the sentence
      # padding = 'max_length'
      pad_to_max_length=True, 
      return_attention_mask=True,#construct attention mask
      return_tensors='pt' #return pytorch tensor
  )
  #add encoded sentence to the list
  input_ids.append(encoded_dict['input_ids'])

  # And its attention mask (simply differentiates padding from non-padding).
  attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
stances = torch.tensor(stances)

# Print sentence 0, now as a list of IDs.
print('Headline: ', headlines[0])
print('Article Body: ',articleBodies[0])
print('Token IDs:', input_ids[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Headline:  Schoolboy almost killed by electric shock claims he now has superpowers like Magneto
Article Body:  'It's a nonsense', says official

North Korea has moved to deny reports its leader Kim Jong-un is planning to open a restaurant in Scotland.

The dictator presides over a chain of restaurants known as “Pyongyang”, with international branches already including Amsterdam, and a number of North Korea experts had speculated that Scotland would be a natural target for expansion.

Kim was reported to have taken a keen interest in Scottish affairs during the referendum debate in September, when officials claimed he felt a vote for independence “would be a very positive thing”.

It has also been noted that Scotch whisky is a favourite tipple among the North Korean elite, while tourists can be asked to make gifts of the drink to tour guides and hosts instead of cash.

Michael Madden, editor of the North Korea Leadership Watch blog, told the Scotsman it “would not surprise me at all if 

In [31]:
from torch.utils.data import TensorDataset, random_split
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, stances)

In [32]:
# 0.7 - training and 0.3- testing
train_size = int(0.7*len(dataset))
test_size = len(dataset)-train_size

# splitting 
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [33]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(train_dataset,sampler = RandomSampler(train_dataset),batch_size = 16)
test_dataloader = DataLoader(test_dataset,sampler = RandomSampler(test_dataset),batch_size = 16)


In [34]:
itr = iter(train_dataloader)
data = itr.next()
print(data)

[tensor([[  101, 18301,  3003,  ...,  1005,  1055,   102],
        [  101, 18301,  7795,  ...,  2028,  3861,   102],
        [  101,  8112,  1010,  ...,     0,     0,     0],
        ...,
        [  101, 18301, 17671,  ...,     0,     0,     0],
        [  101,  2508, 17106,  ...,  1996,  5186,   102],
        [  101, 29584,  2003,  ...,  2010, 13878,   102]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2])]


In [35]:
# optimizer 
from transformers import AdamW
optimizer=AdamW(model.parameters(),lr=5e-5,eps=1e-8)
epochs=2

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [36]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [37]:
print(len(train_dataset))
device = torch.device('cpu')

7000


In [38]:
df.Stance.value_counts()

2    8902
1     891
0     207
Name: Stance, dtype: int64

In [39]:
len(train_dataloader)

438

In [40]:
for epoch_i in range(0,epochs):
  model.train()
  print(epoch_i)
  # print(len(train_dataloader))
  epoch_loss = 0
  for step,batch in enumerate(train_dataloader):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    # print(b_labels)
    print("epoch =",epoch_i,"step =",step,end=" ")
    model.zero_grad() 
    output = model(b_input_ids, 
                            #  token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
    print("loss =",output.loss)
    epoch_loss += output.loss.item()
    output.loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()

  print("Total loss in epoch =",epoch_loss)
  print("avg loss in epoch",epoch_i," =",epoch_loss/len(train_dataloader))


0
epoch = 0 step = 0 loss = tensor(1.0212, grad_fn=<NllLossBackward0>)
epoch = 0 step = 1 loss = tensor(0.8613, grad_fn=<NllLossBackward0>)
epoch = 0 step = 2 loss = tensor(0.6819, grad_fn=<NllLossBackward0>)
epoch = 0 step = 3 loss = tensor(0.6800, grad_fn=<NllLossBackward0>)
epoch = 0 step = 4 loss = tensor(0.4901, grad_fn=<NllLossBackward0>)
epoch = 0 step = 5 loss = tensor(0.2907, grad_fn=<NllLossBackward0>)
epoch = 0 step = 6 loss = tensor(0.6052, grad_fn=<NllLossBackward0>)
epoch = 0 step = 7 loss = tensor(0.3023, grad_fn=<NllLossBackward0>)
epoch = 0 step = 8 loss = tensor(0.4524, grad_fn=<NllLossBackward0>)
epoch = 0 step = 9 loss = tensor(0.4198, grad_fn=<NllLossBackward0>)
epoch = 0 step = 10 loss = tensor(0.1143, grad_fn=<NllLossBackward0>)
epoch = 0 step = 11 loss = tensor(0.0963, grad_fn=<NllLossBackward0>)
epoch = 0 step = 12 loss = tensor(0.5973, grad_fn=<NllLossBackward0>)
epoch = 0 step = 13 loss = tensor(0.0760, grad_fn=<NllLossBackward0>)
epoch = 0 step = 14 loss = t

In [41]:
save_directory = 'saved'
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

In [41]:
from google.colab import drive
drive.mount('/content/gdrive')

model_save_name="bert-stance-model.pt"
path = f'/content/gdrive/My Drive/{model_save_name}'
torch.save(model.state_dict(),path)


In [42]:
#Define a helper function for calculating accuracy.
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def validation_accuracy(orig, pred):
  correct = 0
  agree_correct=0
  disagree_correct=0
  unrelated_correct=0
  total_agree=0
  total_disagree=0
  total_unrelated=0
  for i in range(len(orig)):
    if orig[i]==pred[i]:
      correct+=1
    if orig[i]==1:
      total_agree+=1
      if pred[i]==1:
        agree_correct+=1
    if orig[i]==0:
      total_disagree+=1
      if pred[i]==0:
        disagree_correct+=1 
    if orig[i]==2:
      total_unrelated+=1
      if pred[i]==2:
        unrelated_correct+=1

  if total_agree==0:
    agree_acc = 1
  else:
    agree_acc = agree_correct/total_agree

  if total_disagree==0:
    disagree_acc=1
  else:
    disagree_acc = disagree_correct/total_disagree

  if total_unrelated==0:
    unrelated_acc=1
  else:
    unrelated_acc = unrelated_correct/total_unrelated
  return [correct/len(orig),agree_acc,disagree_acc,total_agree,total_disagree,unrelated_acc,total_unrelated]


In [43]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [44]:
## validation ##

import torch.nn.functional as F
t0 = time.time()
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_agree_accuracy=0
total_disagree_accuracy=0
total_unrelated_accuracy=0
total_eval_loss = 0
nb_eval_steps = 0
total_agree=0
total_disagree=0
total_unrelated=0

# Evaluate data for one epoch
for step,batch in enumerate(test_dataloader):
    
    # Unpack this training batch from our dataloader. 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    with torch.no_grad():        

        print('step =',step)
        output = model(b_input_ids, 
                                # token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        predictions = F.softmax(output.logits,dim=1)
        labels = torch.argmax(predictions,dim=1)
        print("original labels =",b_labels )
        print("labels =",labels)

    acc = validation_accuracy(b_labels, labels)
    print("Accuracy for batch",step+1," =",acc[0])
    total_eval_accuracy += acc[0]
    total_agree_accuracy += acc[1]
    total_disagree_accuracy+=acc[2]
    total_agree+=acc[3]
    total_disagree+=acc[4]
    total_unrelated_accuracy+=acc[5]
    total_unrelated+=acc[6]

    # Accumulate the validation loss.
    l=output.loss.item()
    print("loss =",l)
    total_eval_loss += l

    # Move logits and labels to CPU
    logits = output.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    print()
    print()

    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("Accuracy: {0:.2f}".format(avg_val_accuracy))


# Report the final agree accuracy for this validation run.
avg_agree_accuracy = total_agree_accuracy / total_agree
print("Average Agree Accuracy: {0:.2f}".format(avg_agree_accuracy))

# Report the final disagree accuracy for this validation run.
avg_disagree_accuracy = total_disagree_accuracy / total_disagree
print("Average Disagree Accuracy: {0:.2f}".format(avg_disagree_accuracy))

# Report the final unrelated accuracy for this validation run.
avg_unrelated_accuracy = total_unrelated_accuracy / total_unrelated
print("Average Unrelated Accuracy: {0:.2f}".format(avg_unrelated_accuracy))

# Calculate the average loss over all of the batches.
avg_val_loss = total_eval_loss / len(test_dataloader)
print("Average validation loss =",avg_val_loss)

# Measure how long the validation run took.
validation_time = format_time(time.time() - t0)
print("Validation took: {:}".format(validation_time))


step = 0
original labels = tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2])
labels = tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2])
Accuracy for batch 1  = 1.0
loss = 0.011744881980121136


step = 1
original labels = tensor([2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2])
labels = tensor([2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2])
Accuracy for batch 2  = 1.0
loss = 0.022237958386540413


step = 2
original labels = tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
labels = tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
Accuracy for batch 3  = 1.0
loss = 0.0011289078975096345


step = 3
original labels = tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 1])
labels = tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1])
Accuracy for batch 4  = 0.9375
loss = 0.13522794842720032


step = 4
original labels = tensor([2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2])
labels = tensor([2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2])
Accuracy for ba